In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
# check GPU version
!nvidia-smi

Sun Dec 12 20:39:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/drive/MyDrive/HME/Temporal-Transformer

/content/drive/MyDrive/HME/Temporal-Transformer


In [ ]:
# requirements installation
!pip install einops
!pip install PyPI
!pip install timm
!pip install import-ipynb
# !pip install ipynb

In [ ]:
import import_ipynb
import torch.optim as optim
from main.arguments import parse_args
from main.plot_training_curve import train_plot, test_plot
import numpy as np
import os

args = parse_args()
print(args)

importing Jupyter notebook from /content/drive/MyDrive/HME/Temporal-Transformer/main/arguments.ipynb
importing Jupyter notebook from /content/drive/MyDrive/HME/Temporal-Transformer/main/plot_training_curve.ipynb
Namespace(batch_size=512, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='h36m', dropout=0.0, epochs=200, evaluate='', export_training_curves=False, learning_rate=0.0001, lr_decay=0.99, number_of_frames='/root/.local/share/jupyter/runtime/kernel-635038bf-4216-49e9-8c04-1e780e4e658d.json', resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8')


In [ ]:
%run main/data_generator.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HME/Temporal-Transformer
136
dividing data......
[array([['S6', 'Discussion 1.54138969', 'frame_1.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_21.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_41.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_61.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_81.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_101.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_121.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_141.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_161.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_181.jpg']], dtype='<U21'), array([['S6', 'Discussion 1.54138969', 'frame_201.jpg']], dtype='<U21'), a

In [ ]:
from model import *
# from model import VideoTransformer

importing Jupyter notebook from model.ipynb


In [ ]:
receptive_field = 8
# initialize the model
model_pos_train = VideoTransformer(num_frame=receptive_field, in_chans = 82, embed_dim_ratio = 16, num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None, drop_path_rate=0.1)
model_pos = VideoTransformer(num_frame=receptive_field, in_chans = 82, embed_dim_ratio = 16, num_heads=8, mlp_ratio=2., qkv_bias=True, qk_scale=None, drop_path_rate=0.1)

In [ ]:
# count the trainable paramters
model_params = 0
for parameter in model_pos_train.parameters():
  model_params += parameter.numel()
print('INFO: Trainable paramter count:', model_params)

INFO: Trainable paramter count: 55373051


In [ ]:
# convert the data structure to cuda
if torch.cuda.is_available():
  model_pos_train = nn.DataParallel(model_pos_train)
  model_pos_train = model_pos_train.cuda()
  model_pos = nn.DataParallel(model_pos)
  model_pos = model_pos.cuda()

In [ ]:
# resume
# args.resume = 'epoch_50.bin'   # change this argument to resume the train or start a new training

if args.resume or args.evaluate:
  chk_filename = os.path.join(args.checkpoint, args.resume if args.resume else args.evaluate)      # check point filename
  print('Loading Checkpoint', chk_filename)
  checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
  model_pos_train.load_state_dict(checkpoint['model_pos'], strict=False)
  model_pos.load_state_dict(checkpoint['model_pos'], strict=False)

In [ ]:
%run main/transformation.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HME/Temporal-Transformer
***** LOGDIR*****:   /data/ssd1/russales/logs/07122021-103557
***** LOGDIR*****:   /data/ssd1/russales/logs/07122021-103557


In [ ]:
%cd /content/drive/MyDrive/HME/Temporal-Transformer

/content/drive/MyDrive/HME/Temporal-Transformer


In [ ]:
# %run hmr.2.ipynb


In [ ]:
%run main/loss.ipynb

In [ ]:
# train and validate
## load dataset
print('Loading dataset.....')
dataset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/trainingdata.npy'
# args.dataset should be the name of the npz file
data = np.load(dataset_path, allow_pickle=True)
n_batch = len(data) // args.batch_size
print('number of batches in an epoch: ', n_batch)
dataloader = Dataloader_zzx_central_gt(data, batch_size=512)
args.epochs = 200


# test initialize
testset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/test_data.npy'
# args.dataset should be the name of the npz file

test_data = np.load(testset_path, allow_pickle=True)
n_test_batch = len(test_data) // args.batch_size
dataloader_test = Dataloader_zzx_central_gt(test_data, batch_size=512)
error = 0


# check is epoch is defined
if 'epoch' not in locals():
    epoch = 0   # else it remains the same with 

# lr = args.learning_rate
lr = 0.001

if 'losses' not in locals():
    losses = []   # store all the loss

if 'errors' not in locals():
    errors = []

optimizer = optim.AdamW(model_pos_train.parameters(), lr=lr, weight_decay=0.1)

best_epoch_error = 100


## training
while epoch < args.epochs:
  print('current epoch ', epoch)
  epoch_loss = 0
  error = 0
  model_pos_train.train()
  for j, central_gt in dataloader.generator():
      if torch.cuda.is_available():
          central_gt = central_gt.cuda()
          j = j.cuda()
          # print('yes')
          predict = model_pos_train(j)

      else:
          predict = model_pos_train(j)
      
      optimizer.zero_grad()       # clear all the gradiants before each batch iteration
      
      loss = torch.mean(torch.norm((predict.squeeze() - central_gt.squeeze()), dim = 1))

      loss.backward()

      optimizer.step()

      epoch_loss = epoch_loss + loss

  # end of epoch validation
  with torch.no_grad():
      error = 0
      model_pos.load_state_dict(model_pos_train.state_dict(), strict=False)
      model_pos.eval()
        # change the args definition in argumentation

      for j, central_gt in dataloader_test.generator():
          if torch.cuda.is_available():
              central_gt = central_gt.cuda()
              j = j.cuda()
              # print('yes')
              predict = model_pos_train(j)

          error_batch = torch.mean(torch.norm((predict.squeeze() - central_gt.squeeze()), dim = 1))

          error += error_batch
  
  print('validation error for current epoch is:  ', error/n_test_batch)


  losses.append(epoch_loss/n_batch)
  errors.append(error/n_test_batch)
  !rm Training\ Loss.png
  train_plot(losses)
  test_plot(errors)
  print(epoch_loss/n_batch)
  epoch += 1


  # save the model
# args.checkpoint_frequency = 10
  if best_epoch_error > error/n_test_batch:
      best_epoch_error = error/n_test_batch

      chk_path = os.path.join(args.checkpoint, 'embed_dim16_lr_0.001', 'best_epoch.bin'.format(epoch))
      print('Saving checkpoint to', chk_path)
      torch.save({
          'epoch': epoch,
          'lr': lr,
          'optimizer': optimizer.state_dict(),
          'model_pos': model_pos_train.state_dict(),
          # 'model_traj': model_traj_train.state_dict() if semi_supervised else None,
          # 'random_state_semi': semi_generator.random_state() if semi_supervised else None,
      }, chk_path)
  if epoch % args.checkpoint_frequency == 0:
      chk_path = os.path.join(args.checkpoint, 'embed_dim16_lr_0.001', 'epoch_{}.bin'.format(epoch))
      print('Saving checkpoint to', chk_path)

      torch.save({
          'epoch': epoch,
          'lr': lr,
          'optimizer': optimizer.state_dict(),
          'model_pos': model_pos_train.state_dict(),
          # 'model_traj': model_traj_train.state_dict() if semi_supervised else None,
          # 'random_state_semi': semi_generator.random_state() if semi_supervised else None,
      }, chk_path)

Loading dataset.....
number of batches in an epoch:  142
current epoch  0
dividing data......
dividing data......
validation error for current epoch is:   tensor(1.5883, device='cuda:0')
rm: cannot remove 'Training Loss.png': No such file or directory
tensor(2.4518, device='cuda:0', grad_fn=<DivBackward0>)
Saving checkpoint to checkpoint/embed_dim16_lr_0.001/best_epoch.bin
current epoch  1
dividing data......
dividing data......
validation error for current epoch is:   tensor(1.2062, device='cuda:0')
tensor(1.3684, device='cuda:0', grad_fn=<DivBackward0>)
Saving checkpoint to checkpoint/embed_dim16_lr_0.001/best_epoch.bin
current epoch  2
dividing data......
dividing data......
validation error for current epoch is:   tensor(1.0737, device='cuda:0')
tensor(1.1508, device='cuda:0', grad_fn=<DivBackward0>)
Saving checkpoint to checkpoint/embed_dim16_lr_0.001/best_epoch.bin
current epoch  3
dividing data......
dividing data......
validation error for current epoch is:   tensor(1.0634, dev

In [ ]:
# train with 3D GT
# from main.loss import loss_compute
import torch
## load dataset
print('Loading dataset.....')
dataset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/trainingdata.npy'
# args.dataset should be the name of the npz file
data = np.load(dataset_path, allow_pickle=True)
n_batch = len(data) // args.batch_size
print('number of batches in an epoch: ', n_batch)
dataloader = Dataloader_zzx_central_hm36_gt_tag(data, batch_size=512)
args.epochs = 200


# test initialize
testset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/test_data.npy'
# args.dataset should be the name of the npz file

test_data = np.load(testset_path, allow_pickle=True)
n_test_batch = len(test_data) // args.batch_size
dataloader_test = Dataloader_zzx_central_hm36_gt_tag(test_data, batch_size=512)
error = 0


# check is epoch is defined
if 'epoch' not in locals():
    epoch = 0   # else it remains the same with 

lr = args.learning_rate
# lr = 0.0001

if 'losses' not in locals():
    losses = []   # store all the loss

if 'errors' not in locals():
    errors = []

optimizer = optim.AdamW(model_pos_train.parameters(), lr=lr, weight_decay=0.1)

best_epoch_error = 10000000


## training
while epoch < args.epochs:
  print('current epoch ', epoch)
  epoch_loss = 0
  epoch_loss1 = 0
  epoch_loss2 = 0
  error = 0
  model_pos_train.train()
  for j, central_gt, gt_3d, cams, name in dataloader.generator():
      if torch.cuda.is_available():
          central_gt = central_gt.cuda()
          j = j.cuda()
          # print('yes')
          predict = model_pos_train(j)

      else:
          predict = model_pos_train(j)
      
      optimizer.zero_grad()       # clear all the gradiants before each batch iteration
      
      loss, loss1, loss2 = loss_compute_grad(predict, central_gt, gt_3d, epoch)
      
      
      loss.backward()

      optimizer.step()

      epoch_loss = epoch_loss + loss
      epoch_loss1 = epoch_loss1 + loss1
      epoch_loss2 = epoch_loss2 + loss2
  
  print(" The total loss is: ", epoch_loss/n_batch, '\n', 'The smpl loss is: ', epoch_loss1/n_batch, '\n', 'The 3D joints loss is: ', epoch_loss2/n_batch)

  # end of epoch validation
  with torch.no_grad():
      error = 0
      model_pos.load_state_dict(model_pos_train.state_dict(), strict=False)
      model_pos.eval()
        # change the args definition in argumentation

      for j, central_gt, gt_3d, cams, name in dataloader_test.generator():
          if torch.cuda.is_available():
              central_gt = central_gt.cuda()
              j = j.cuda()
              # print('yes')
              predict = model_pos_train(j)

          # error_batch = torch.mean(torch.norm((predict.squeeze() - central_gt.squeeze()), dim = 1))
          error_batch = mpjpe(predict, central_gt, gt_3d)

          error += error_batch
  
  print('validation error for current epoch is:  ', error/n_test_batch)


  losses.append(epoch_loss/n_batch)
  errors.append(error/n_test_batch)
  !rm Training\ Loss.png
  train_plot(losses)
  test_plot(errors)
  epoch += 1


  # save the model
# args.checkpoint_frequency = 10
  if best_epoch_error > error/n_test_batch:
      best_epoch_error = error/n_test_batch

      chk_path = os.path.join(args.checkpoint, 'embed_3D_growth_dim16_lr_0.001', 'best_epoch.bin'.format(epoch))
      print('Saving checkpoint to', chk_path)
      torch.save({
          'epoch': epoch,
          'lr': lr,
          'optimizer': optimizer.state_dict(),
          'model_pos': model_pos_train.state_dict(),
          # 'model_traj': model_traj_train.state_dict() if semi_supervised else None,
          # 'random_state_semi': semi_generator.random_state() if semi_supervised else None,
      }, chk_path)
  if epoch % args.checkpoint_frequency == 0:
      chk_path = os.path.join(args.checkpoint, 'embed_3D_growth_dim16_lr_0.001', 'epoch_{}.bin'.format(epoch))
      print('Saving checkpoint to', chk_path)

      torch.save({
          'epoch': epoch,
          'lr': lr,
          'optimizer': optimizer.state_dict(),
          'model_pos': model_pos_train.state_dict(),
          # 'model_traj': model_traj_train.state_dict() if semi_supervised else None,
          # 'random_state_semi': semi_generator.random_state() if semi_supervised else None,
      }, chk_path)

In [ ]:
## visualization for one specific video


In [ ]:
# visualization
# render the smpl model with the prediction output
index = 0
dataloader = Dataloader_zzx_central_gt_frame(data, batch_size=1)

# load the data for only one batch
for sample, central, name in dataloader.generator():
    print(sample.shape)
    break

with torch.no_grad():
      error = 0
      model_pos.load_state_dict(model_pos_train.state_dict(), strict=False)
      model_pos.eval()
        # change the args definition in argumentation
      if torch.cuda.is_available():
          central = central.cuda()
          sample = sample.cuda()
          # print('yes')
          predict = model_pos(sample)

print(predict.shape)
# the shape is correct

shape_and_pose = predict.cpu().detach().numpy()

central = central.cpu().detach().numpy()

# convert torch.cuda to numpy
## convert the predicted pose_and_shape to vertices
shape_and_pose = np.expand_dims(shape_and_pose.squeeze(), axis = 0)
central = np.expand_dims(central.squeeze(), axis = 0)

vertices, joints_3d, shapes = smpl2vertices(shape_and_pose)
vertices_gt, joints_3d_gt, shapes_gt = smpl2vertices(central)

vertices = vertices.numpy()
vertices_gt = vertices_gt.numpy()
# print(type(vertices))
vertices = vertices.squeeze()
vertices_gt = vertices_gt.squeeze()

np.save('vertices', vertices)
np.save('vertices_gt', vertices_gt)
np.save('name', name)
print(name)

In [ ]:
# check the prediction and central input

# print(central[0], '\n', predict[0])
print(predict[0,0,0])
print(predict)
print(central_gt)
print(predict[0].squeeze() - central_gt[0].squeeze())
# torch.norm((predict[0].squeeze() - central_gt[0].squeeze()), dim = 1)

In [ ]:
# plot the training loss

train_plot(losses)
test_plot(errors)

In [ ]:
min(errors)

# **Some modules that can be used**

In [ ]:
## load dataset
print('Loading dataset.....')
dataset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/trainingdata.npy'
# args.dataset should be the name of the npz file

data = np.load(dataset_path, allow_pickle=True)
n_batch = len(data) // args.batch_size
print('number of batches in an epoch: ', n_batch)
dataloader = Dataloader_zzx_central_gt(data, batch_size=512)
args.epochs = 200

# check is epoch is defined
if 'epoch' not in locals():
    epoch = 0   # else it remains the same with 

lr = args.learning_rate

if 'losses' not in locals():
    losses = []   # store all the loss

optimizer = optim.AdamW(model_pos_train.parameters(), lr=lr, weight_decay=0.1)


## training
while epoch < args.epochs:
  print('current epoch ', epoch)
  epoch_loss = 0
  model_pos_train.train()
  for j, central_gt in dataloader.generator():
      if torch.cuda.is_available():
          central_gt = central_gt.cuda()
          j = j.cuda()
          # print('yes')
          predict = model_pos_train(j)

      else:
          predict = model_pos_train(j)
      
      optimizer.zero_grad()       # clear all the gradiants before each batch iteration
      
      loss = torch.mean(torch.norm((predict.squeeze() - central_gt.squeeze()), dim = 1))

      loss.backward()

      optimizer.step()

      epoch_loss = epoch_loss + loss
  
  losses.append(epoch_loss/n_batch)
  !rm Training\ Loss.png
  train_plot(losses)
  print(epoch_loss/n_batch)
  epoch += 1

In [ ]:
# test the model
testset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/test_data.npy'
# args.dataset should be the name of the npz file

test_data = np.load(testset_path, allow_pickle=True)
n_test_batch = len(test_data) // args.batch_size
dataloader_test = Dataloader_zzx_central_gt(test_data, batch_size=512)
error = 0

with torch.no_grad():
    model_pos.load_state_dict(model_pos_train.state_dict(), strict=False)
    model_pos.eval()
      # change the args definition in argumentation

    for j, central_gt in dataloader_test.generator():
        if torch.cuda.is_available():
            central_gt = central_gt.cuda()
            j = j.cuda()
            # print('yes')
            predict = model_pos_train(j)

        error_batch = torch.mean(torch.norm((predict.squeeze() - central_gt.squeeze()), dim = 1))
        # print(error_batch)
        error += error_batch

print(error/n_test_batch)

In [ ]:
# visualization


In [ ]:
## convert predicted 'shape_and_pose' to 3D joints: check the number
from main import model_util
from main.config import Config
from main.smpl import Smpl
import numpy as np

print(predict.shape)

# use the smpl function
smpl = Smpl()

# vertices, joints_3d, rotations = smpl(pose_and_shape, **kwargs)

In [ ]:
dataset_path = '/content/drive/MyDrive/HME/Temporal-Transformer/trainingdata.npy'
# args.dataset should be the name of the npz file

data = np.load(dataset_path, allow_pickle=True)
# data = data.transpose()
len(data) // 512

In [ ]:
A = torch.ones(2,1,3)
A
B = torch.zeros(2,1,3)
torch.mean(torch.norm((A.squeeze() - B.squeeze()), dim = 1))

In [ ]:
predict.shape

In [ ]:
# don't know if uses?

def eval_data_prepare(receptive_field, inputs_2d, inputs_3d):
  inputs_2d_p = torch.squeeze(inputs_2d)
  inputs_3d_p = inputs_3d.permute(1, 0, 2, 3)
  out_num = inputs_2d_p.shape[0] - receptive_field + 1
  eval_input_2d = torch.empty(out_num, receptive_field, inputs_2d_p.shape[1], inputs_2d_p.shape[2])
  for i in range(out_num):
    eval_input_2d[i, :, :, :] = inputs_2d_p[i: i + receptive_field, :, :]
    return eval_input_2d, inputs_3d_p


In [ ]:
# load the training data using ChunkedGenerator
# cameras_train, poses_train, poses_train_2d = fetch(subjects_train, action_filter, subset = args.subset)
# fetch function is defined above

lr = args.learning_rate
optimizer = optim.AdamW(model_pos_train.parameters(), lr = lr, weight_decay=0.1)

lr_decay = args.lr_decay
losses_3d_train = []
losses_3d_train_eval = []
losses_3d_valid = []

epoch = 0
initial_momentum = []
final_momentum = []

train_generator = ChunkedGenerator(args.batch_size, camera_train, poses_train, pose_train_2d, args.stride,
                                    pad = pad, causal_shift = causal_shift, shuffle = True, augment=args.data_augmentation,
                                    kps_left=kps_left, kps_right=kps_right, joints_left=joints_left, joints_right=joints_right)

train_generator_eval = UnchunkedGenerator(cameras_train,poses_train,poses_train_2d,
                                          pad=pad, causal_shift=causal_shift, augment=False)

print('INFO: Training on {} frames'.format(train_generator_eval.num_frames()))

if args.resume:
  epoch = checkpoint['epoch']
  if 'optimizer' in checkpoint and checkpoint['optimizer'] in not None:
    optimizer.load_state_dict(checkpoint['optimizer'])
    train_generator.set_random_state(checkpoint['random_state'])
  
  else:
    print('WARNING: this checkpoint does not contain an optimizer state. the optimizer will be reinitialized.')

  lr = checkpoint['lr']

print('** Note: reported losses are averaged over all frames.')
print('** The final evaluation will be carried out after the last training epoch')

In [ ]:
# training
while epoch < args.epochs:
    print('current epoch:   ', epoch)
    start_time = time()
    epoch_loss_3d_train = 0
    epoch_loss_traj_train = 0
    epoch_loss_2d_train_unlabeled = 0
    N = 0
    N_semi = 0
    model_pos_train.train()       # tell the model that it's traing, some of the layers would function differently

    for cameras_train, batch_3d, batch_2d, smpl in train_generator.next_epoch():
      cameras_train = torch.from_numpy(camera_train.astype('float32'))
      inputs_3d = torch.from_numpy(batch_3d.astype('float32'))
      inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
      smpl = torch.from_numpy(smpl.astype('float32'))

      if torch.cuda.is_available():
          inputs_3d = inputs_3d.cuda()
          inputs_2d = inputs_2d.cuda()
          cameras_train = cameras_train.cuda()
      inputs_traj = inputs_3d[:, :, :1].clone()
      inputs_3d[:, :, 0] = 0

      optimizer.zero_grad()

      # Predict 3D poses
      predicted_3d_pos = model_pos_train(inputs_2d)
      
      del inputs_2d_p
      torch.cuda.empty_cache()

      loss_3d_poses = mpjpe(predicted_3d_pos, inputs_3d)
      epoch_loss_3d_train += inputs_3d.shape[0] * inputs_3d.shape[1] * loss_3d_pos.item()
      N += inputs_3d.shape[0] * inputs_3d.shape[1]

      loss_total = loss_3d_pos

      loss_total.backward()

      optimizer.step()

      del inputs_3d, loss_3d_pos, predicted_3d_pos
      torch.cuda.empty_cache()
    
    loss_3d_train.append(epoch_loss_3d_train / N)

    # print the result in every epoch
    print('[%d] time %.2f lr % f 3d_train %f' %(
        epoch + 1,
        elapsed,
        lr,
        losses_3d_train[-1]     # loss in nearest epoch
    ))


**Lack of evaluation within epochs, and the validation after the training is done.**

In [ ]:
# save the model

# args.checkpoint_frequency = 10
if epoch % args.checkpoint_frequency == 0:
    chk_path = os.path.join(args.checkpoint, 'epoch_{}.bin'.format(epoch))
    print('Saving checkpoint to', chk_path)

    torch.save({
        'epoch': epoch,
        'lr': lr,
        'optimizer': optimizer.state_dict(),
        'model_pos': model_pos_train.state_dict(),
        # 'model_traj': model_traj_train.state_dict() if semi_supervised else None,
        # 'random_state_semi': semi_generator.random_state() if semi_supervised else None,
    }, chk_path)

In [ ]:
# clear all the variables

%reset